# Block Group Census Population Download for Trimet Residential Analysis

### This script downloads census data exports county level data to an excel file and tract level data to shapefiles.

## TO DO 
- Methodize Scripts


In [1]:
# Import necessary libraries

import requests
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
import os


In [2]:
# FIPS CODES
Multnomah = "051"
Oregon ="41"

# For each iteration change the county
county = Multnomah
state = Oregon

In [8]:
# Block Group Geographies
block_groups_shp = "/Users/calvindechicago/Documents/GitHub/JWA_2021/cartographic_tracts_oregon/cb_2018_41_bg_500k/cb_2018_41_bg_500k.shp"
print("Census Block Group Geographies filepath is correct: ", os.path.isfile(census_tracts_shp))


# Set Census Geography level (select all tracts or all block groups) 
# This will be used throughout code to pull either tract or block group data
census_geo_string = "block group"
#census_geo_string = "tract"

#Set output filepaths
Equity_Tracts = f'/Users/calvindechicago/Documents/GitHub/JWA_2021/code_outputs/equity_tracts_{county}.gpkg'
BG_Pop = f'/Users/calvindechicago/Documents/GitHub/JWA_2021/code_outputs/bg_pop_{county}.gpkg'
EXCEL_FP = f'/Users/calvindechicago/Documents/GitHub/JWA_2021/code_outputs/countylevel_demographics_{county}.xlsx'

Census Block Group Geographies filepath is correct:  True


In [9]:
# These are all of the main variables used to build a call url to the the census api website
# Available APIs (--> 2018 ACS Detailed Tables Variables [ html | xml | json ])
# https://www.census.gov/data/developers/data-sets.html
# SETUP FOR TABLE B01003: Total Population
# Create Block Group Populations
# TABLE: B01003_001E

HOST = "https://api.census.gov/data"
year = "2019"
dataset = "acs/acs5"
api_key = "f9e79198302081250c07d556f35d8a81cdae528a"
base_url = "/".join([HOST, year, dataset,])

In [10]:
# BUILD API REQUEST 
# Only 50 variables per request are permitted.

request_predicates = {}
get_vars_age = ["NAME", "B01003_001E","GEO_ID"]

col_bg_pop = ["name_string","total_pop","geoid","state", "county", "tract", "block_group"]

request_predicates["key"] = api_key
request_predicates["get"] = ",".join(get_vars_age)

#THESE PREDICATES GET DATA FOR EVERY BLOCK GROUP IN COUNTY

request_predicates["for"] = f"block group:*"
request_predicates["in"] = f"state:{state}+county:{county}"

# Build Census API Request
bg_pop = requests.get(base_url, params=request_predicates)


In [11]:
#Setting up Block Group Population data frame, getting rid of first header row
df_bg_pop = pd.DataFrame(columns=col_bg_pop, data=bg_pop.json()[1:])


In [12]:
#The geoid field in the df_transpo_mode table does not match the Tigerlines geoid field. 
#This slices the the right 11 most digits, which match the geoid codes in the TigerLine file. 
#(... these are state ('06') for California, followed by county, followed by census tract)
num_columns = df_bg_pop.shape[1]

df_bg_pop.insert(num_columns, "geoid_join", df_bg_pop['geoid'].str.slice(start=9), True) 


In [13]:
# Join the census data to Tigerline Cartographic Boundary block group geometries. 
# Census Tract Tigerline California block group


#This reads the census tracts shapefile into a geodataframe
bg_gdf = gpd.read_file(block_groups_shp)

#Make tigerline boundary columns lowercase
bg_gdf.columns = map(str.lower, bg_gdf.columns)

In [14]:
# Filter to only include Multnomah County
bg_gdf = bg_gdf[bg_gdf['countyfp']=='051']

In [15]:
# Merge block group geographies data with the population data produced in earlier steps
final_bg_pop = bg_gdf.merge(df_bg_pop,left_on='affgeoid',right_on='geoid')

In [ ]:
# Export File
final_bg_pop.to_file(BG_Pop, layer='BG_Pop', driver="GPKG")